# Quantum Computing Workshop: Introduction to Qiskit 1

In [ ]:
# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, transpile
from qiskit.visualization import *
from qiskit_aer import AerSimulator, Aer
from qiskit.quantum_info import Operator, Statevector


from qiskit_ibm_runtime import QiskitRuntimeService, Sampler, Estimator, Session, Options

# Loading your IBM Quantum account(s)
with open('api_key.txt', 'r') as file:
    token = file.read()

service = QiskitRuntimeService(channel="ibm_quantum",token=token)

In [ ]:
# Other useful imports
import numpy as np
# Ignore future warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

## Backends

- **Simulators:** Based on classical hardware. Great for prototyping and learning
    - Local
        - AerSimulator: Simulator for quantum circuit execution.
        - Statevector simulator: Reveals information on the full state of the system of qubits.
    - Cloud
        - qasm_simulator: Can mimic hardware noise.
- **Real Quantum hardware:** Used when execution on simulators does not scale.

In [ ]:
# Instantiate a backend for local execution
aer_backend = AerSimulator()
# Instantiate a statevector simulator backend
sv_backend = Aer.get_backend("statevector_simulator")

## Basic Not circuit
Goal: flip a qubit from the initial $|0\rangle$ state to the $|1\rangle$ state

Note that qubits on IBM systems are always reset to the $|0\rangle$ state at the start of the circuit.

In [ ]:
# Create a NOT circuit

# Your code goes here


**Note**: 

Every circuit performs a transformation to our initial state of qubits. 

This transformation can be represented as matrix.

In [ ]:
# Display the Operator (2x2 Matrix) that corresponds to this circuit

# Your code goes here

In [ ]:
# Measure qbit 0 -> cbit 0

# Your code goes here

In [ ]:
# Run the circuit on a local backend with different number of shots (1,10, 2048)
# What are the respective counts?

# Your code goes here

In [ ]:
# Visualize the counts

# Your code goes here

**Note**:

No matter how repeatedly we execute the NOT circuit, the result is always the same: This circuit performs a deterministic operation (if we omit the noise, present in real quantum hardware).

## Superposition

Goal: place a qubit into an equal superposition of $|0\rangle$ and $|1\rangle$

In [ ]:
# Create superposition circuit (sp_circ) by using the Hadamard gate. Measure qbit 0 -> cbit 0

# Your code goes here

In [ ]:
# Run the simulation for different number of shots (1,10, 2048)
# What are the respective counts?
job = aer_backend.run(sp_circ,shots=1)
sp_counts = job.result().get_counts()
sp_counts

In [ ]:
# Plot
plot_histogram(sp_counts)

**Note**:

The Hadamard operation is probabilistic. Classical computers can only simulate this operation. Real quantum computers can execute probabilistic operations natively.

## Multiple qubits in superposition

In [ ]:
# Create superposition circuit (sp_circ2) between 3 qubits and measure them.
n_qubits = 3

# Your code goes here

In [ ]:
# Run the superpositon circuit on a statevector simulator backend
# Visualize the states on bloch spheres

# Your code goes here

In [ ]:
sp_circ2.measure_all()
sp_circ2.draw()

In [ ]:
# Run simulation and plot results
job = aer_backend.run(sp_circ2,shots=2**14)
sp_counts = job.result().get_counts()
for state, count in sp_counts.items():
    print(f'State: {state} - Measurement probability: {round(count/sum(sp_counts.values()),4)}')
plot_histogram(sp_counts)

**Questions**:
1) When creating a superposition between N-Qubits, how many possible states can be measured? 
2) What is the probability of measuring one particular state if all qubits are in equal superposition?

## Bell state (entanglement)
Goal: Create the $\frac{1}{\sqrt{2}} (|00\rangle + |11\rangle)$ entangled state (Bell state)

When measuring one qubit it is with equal probability in the $|0\rangle$ or $|1\rangle$ state.

The measurement of one qubit "collapses" the other qubit.

The measurements among maximally entangled qubits are perfectly correlated.

In [ ]:
# Create Bell state circuit (bell_circ). Meaure all qubits to clbits.

# Your code goes here

In [ ]:
# Run the bell circuit on a local backend and extract the counts (bell_counts)

# Your code goes here

In [ ]:
plot_histogram(bell_counts)

## Running on a real quantum computer

In [ ]:
# List all backends that are available for you

# Your code goes here

In [ ]:
# Simpliy select the least busy backend

# Your code goes here

In [ ]:
# Or select a particular backend

# Your code goes here

In [ ]:
print(f"Native gates: {ibmq_backend.operation_names}")

**Note**:

The job failed because the Hadamard gate is not supported as native gate instruction on the selected backend.

We have to **transpile** the circuit first: Translate the gates in our circuit to natively supported gates.

In [ ]:
print(f'Original circuit depth {bell_circ.depth()}')
bell_circ_tp = transpile(bell_circ, ibmq_backend)
print(f'Transpiled circuit depth {bell_circ_tp.depth()}')
bell_circ_tp.draw(idle_wires=False)

**Note**

Transpilation usually increases the depth of our circuit.

In [ ]:
job_bell_tp = ibmq_backend.run(bell_circ_tp)
job_bell_tp

In [ ]:
job_bell_tp.status()

In [ ]:
service.jobs()

In [ ]:
job_bell_tp_1 = service.job("csrsvsfvkv50008g94y0")
job_bell_tp_1.logs()

In [ ]:
counts = job_bell_tp_1.result().get_counts()
plot_histogram(counts)

**Note**:

The results on real quantum hardware shows measurement probabilities for the $|01\rangle$ and $|10\rangle$ states. This is due to **noise** (inaccuracies) in the quantum hardware.

## GHZ Circuit

A GHZ state is a maximal entangled state of multiple qubits, i.e. an extension of the Bell states to more than 2 qubits.

In [ ]:
# Create a 3-qubit GHZ state circuit (ghz_circ)

# Your code goes here

In [ ]:
# Run in simulator and plot out the measured result
ghz_state = sv_backend.run(ghz_circ).result().get_statevector()
ghz_probs = Statevector(ghz_state).probabilities()
ghz_probs

In [ ]:
# visualize the GHZ state on a Q-Sphere

# Your code goes here

## Larger GHZ circuit

Let's take this to the next level and create an even larger GHZ state.

In [ ]:
n_qubits = 8

big_ghz = QuantumCircuit(n_qubits)
big_ghz.h(0)
for i in range(n_qubits-1):
    big_ghz.cx(i, i+1)
big_ghz.measure_all()
big_ghz.draw()

In [ ]:
print(f'Original big GHZ circuit depth {big_ghz.depth()}')
big_ghz_tp = transpile(big_ghz, ibmq_backend, optimization_level=3)
print(f'Transpiled circuit depth {big_ghz_tp.depth()}')
big_ghz_tp.draw(idle_wires=False)

In [ ]:
plot_circuit_layout(big_ghz_tp,ibmq_backend)

In [ ]:
job_big_ghz = ibmq_backend.run(big_ghz_tp)
job_big_ghz

In [ ]:
job_big_ghz_1 = service.job("csrt0fszsqjg008t5thg")
counts = job_big_ghz_1.result().get_counts()
plot_histogram(counts)

## Challenge

The circuit above is simple, but the overall depth of the circuit is long. This happened because the sequence of CNOT gates is serialized. The system can only execute CNOT gates on a particular qubit one at a time. Long circuit depth can be problematic as a longer circuit is more susceptible to noise and decoherence in current hardware. It would be beneficial to create the same state, but with less overall depth.

Can you find a way to reduce the depth of the GHZ circuit?

In [ ]:
# Your code goes here

In [ ]:
# These are your instructors results. 
job_big_ghz_1_1 = service.job("csrt5jyjkdzg00885ah0")
counts = job_big_ghz_1_1.result().get_counts()
plot_histogram(counts)